#### Initial data exploration and prep for `main.py`

In [1]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd
import psycopg

In [2]:
load_dotenv()

True

PDF link to CTA API documentation: https://www.transitchicago.com/assets/1/6/cta_Train_Tracker_API_Developer_Guide_and_Documentation.pdf

* will use the _locations API_ to track multiple locations at once
* produces a list of in service trains w basic info
* 193 [double check] total CTA stops maybe one dict per line
    * line_name = {cta_train_stop_key_id: friendltrain_name}
* json response most important
    * ctatt: root element
    * tmst: time stamp
    * train: container element one per train in response
    * destNm: friendly destination description
    * nextStpId: next stop ID matcing GTFS
    * nextStaId: next station ID matching GTFS
    * nextStaNm: proper name of next station
* do another response request for massive list of parent stop IDs {statId:StatNm}
    * parent stop IDs are for the station overall
    * each direction at the station has it's own ID for additional detail not needed for this project
* create another dictionary that maps StatId to gio pin
* link to total number of stations per train line: https://rtams.org/services/cta/lines
* {parent_station_id: GPIO_PIN}

In [ ]:
CTA_API_KEY = os.getenv('CTA_API_KEY')

rt_names = ["Red", "Blue", "Brn", "G", "Org", "P", "Pink", "Y"]


location_url = ''.join(['http://lapi.transitchicago.com/api/1.0/ttpositions.aspx?key=',
                            CTA_API_KEY, 
                            '&rt=', rt_names[0],
                            '&rt=', rt_names[1],
                            '&rt=', rt_names[2],
                            '&rt=', rt_names[3],
                            '&outputType=JSON'])


# location_url = ''.join(['http://lapi.transitchicago.com/api/1.0/ttpositions.aspx?key='
#                                 ,CTA_API_KEY
#                                 ,'&rt=', rt_names[-4]
#                                 ,'&rt=', rt_names[-3]
#                                 ,'&rt=', rt_names[-2]
#                                 ,'&rt=', rt_names[-1]
#                                 ,'&outputType=JSON'])


response = requests.get(location_url).json()

print(response)

locations_df = pd.DataFrame()

df_cols = ['next_station_parent_id',
           'next_stop_id', 'arrival_time',
           'prediction_time',
           'is_app', 'is_delayed' ]

stop = 0
train = 0


all_nxt_sta_p_id = []
all_nxt_stop_id = []
all_arrival_times = []
all_pred_times = []
all_is_app = []
all_is_dly = []


# list of lists to build df
all_lapi_trn_data = []
all_nxt_sta_dict= {}

rt_len = len(response['ctatt']['route'])
for train in range(rt_len):
        for stop in range(len(response['ctatt']['route'][train]['train'])):
                all_nxt_sta_dict[response['ctatt']['route'][train]['train'][stop]['nextStaId']] = response['ctatt']['route'][train]['train'][stop]['nextStaNm']
                 # repeat this line a lot maybe build funtion for it
                # all_nxt_sta_names.append(response['ctatt']['route'][train]['train'][stop]['nextStaNm'])
                # def append_data(append_list: list, append_col: col) -> list
                all_nxt_sta_p_id.append(response['ctatt']['route'][train]['train'][stop]['nextStaId'])
                all_nxt_stop_id.append(response['ctatt']['route'][train]['train'][stop]['nextStpId'])
                all_arrival_times.append(response['ctatt']['route'][train]['train'][stop]['arrT'])
                all_pred_times.append(response['ctatt']['route'][train]['train'][stop]['prdt'])
                all_is_app.append(response['ctatt']['route'][train]['train'][stop]['isApp'])
                all_is_dly.append(response['ctatt']['route'][train]['train'][stop]['isDly'])

                stop+=1
        train+=1

all_lapi_trn_data.append(all_nxt_sta_p_id)
all_lapi_trn_data.append(all_nxt_stop_id )
all_lapi_trn_data.append(all_arrival_times)
all_lapi_trn_data.append(all_pred_times)
all_lapi_trn_data.append(all_is_app)
all_lapi_trn_data.append(all_is_dly)

lapi_df = pd.DataFrame(dict(zip(df_cols, all_lapi_trn_data)))

print(all_nxt_sta_dict)

{'ctatt': {'tmst': '2026-02-27T23:23:57', 'errCd': '0', 'errNm': None, 'route': [{'@name': 'red', 'train': [{'rn': '834', 'destSt': '30173', 'destNm': 'Howard', 'trDr': '1', 'nextStaId': '40910', 'nextStpId': '30177', 'nextStaNm': '63rd', 'prdt': '2026-02-27T23:23:37', 'arrT': '2026-02-27T23:24:37', 'isApp': '1', 'isDly': '0', 'flags': None, 'lat': '41.77498', 'lon': '-87.6271', 'heading': '321'}, {'rn': '843', 'destSt': '30173', 'destNm': 'Howard', 'trDr': '1', 'nextStaId': '40100', 'nextStpId': '30020', 'nextStaNm': 'Morse', 'prdt': '2026-02-27T23:23:13', 'arrT': '2026-02-27T23:25:13', 'isApp': '0', 'isDly': '0', 'flags': None, 'lat': '42.00836', 'lon': '-87.66591', 'heading': '358'}, {'rn': '844', 'destSt': '30173', 'destNm': 'Howard', 'trDr': '1', 'nextStaId': '40340', 'nextStpId': '30066', 'nextStaNm': 'Berwyn', 'prdt': '2026-02-27T23:23:40', 'arrT': '2026-02-27T23:24:40', 'isApp': '1', 'isDly': '0', 'flags': None, 'lat': '41.97345', 'lon': '-87.65853', 'heading': '358'}, {'rn': '

In [5]:
lapi_df.head()

,next_station_parent_id,next_stop_id,arrival_time,prediction_time,is_app,is_delayed
0,40910,30177,2026-02-27T23:24:37,2026-02-27T23:23:37,1,0
1,40100,30020,2026-02-27T23:25:13,2026-02-27T23:23:13,0,0
2,40340,30066,2026-02-27T23:24:40,2026-02-27T23:23:40,1,0
3,40630,30121,2026-02-27T23:25:31,2026-02-27T23:23:31,0,0
4,41000,30193,2026-02-27T23:24:30,2026-02-27T23:23:30,1,0


In [ ]:
# first 0 is train second 0 is stop
print(response['ctatt']['route'][0]['train'][0])

{'rn': '717', 'destSt': '30182', 'destNm': 'Midway', 'trDr': '5', 'nextStaId': '40930', 'nextStpId': '30182', 'nextStaNm': 'Midway', 'prdt': '2026-02-26T21:57:57', 'arrT': '2026-02-26T21:58:57', 'isApp': '1', 'isDly': '0', 'flags': None, 'lat': '41.79788', 'lon': '-87.72957', 'heading': '248'}


In [4]:
# data downloaded from 
# https://data.cityofchicago.org/Transportation/CTA-System-Information-List-of-L-Stops/8pix-ypme/about_data

metadata_df = pd.read_csv('CTA_List_of_\'L\'_Stops_20260227.csv')
metadata_df[metadata_df['STOP_ID'] == 30182]


,STOP_ID,DIRECTION_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA,RED,BLUE,G,BRN,P,Y,Pnk,O,Location
210,30182,S,Midway (Arrival),Midway,Midway (Orange Line),40930,True,False,False,False,False,False,False,False,True,"(41.78661, -87.737875)"


In [18]:
lapi_df['next_stop_id'] = lapi_df['next_stop_id'].astype(int)

joined_df = lapi_df.merge(metadata_df[['STOP_ID', 'STOP_NAME', 'STATION_DESCRIPTIVE_NAME', 'DIRECTION_ID']],
                          left_on='next_stop_id', right_on='STOP_ID', how='left', suffixes=["_fact", "_dim"]).head()

cols =['next_stop_id', 'STOP_NAME',
       'next_station_parent_id',
       'STATION_DESCRIPTIVE_NAME',
       'arrival_time', 'prediction_time',
       'is_app', 'is_delayed',
       'DIRECTION_ID']

joined_df[cols]

,next_stop_id,STOP_NAME,next_station_parent_id,STATION_DESCRIPTIVE_NAME,arrival_time,prediction_time,is_app,is_delayed,DIRECTION_ID
0,30177,63rd (Howard-bound),40910,63rd (Red Line),2026-02-27T23:24:37,2026-02-27T23:23:37,1,0,N
1,30020,Morse (Howard-bound),40100,Morse (Red Line),2026-02-27T23:25:13,2026-02-27T23:23:13,0,0,N
2,30066,Berwyn (Howard-bound),40340,Berwyn (Red Line),2026-02-27T23:24:40,2026-02-27T23:23:40,1,0,N
3,30121,Clark/Division (Howard-bound),40630,Clark/Division (Red Line),2026-02-27T23:25:31,2026-02-27T23:23:31,0,0,N
4,30193,Cermak-Chinatown (Howard-bound),41000,Cermak-Chinatown (Red Line),2026-02-27T23:24:30,2026-02-27T23:23:30,1,0,N


In [3]:
postgres_password= os.getenv('POSTGRES_PASSWORD')


conn = psycopg.connect(
                dbname="cta_data",
                user="postgres",
                password=postgres_password,
                host="localhost")

#### Col retrieval from lapi
Goal:
* set up local postgres database to store train tracker info
* two dataframes
    * one that will have _all_ station ids, station names, and useful metadata
    * second for storing info from the api
    * key of dictionary is col name value is array that will populate column
        * `dict(zip(my_list_1, my_list_2))`

        * next parent station id
            * parent stop id [non directional]
            * `response['ctatt']['route'][0]['train'][0]['nextStaId']`
        * stop id
            * not parent id
            * ex 30219 
            * join on stop id since stop id is unique and parent id is not
            * `response['ctatt']['route'][0]['train'][0]['nextStpId']`
        * station arrival time
            * prediction of train arrival time
            * `response['ctatt']['route'][0]['train'][0]['arrT']`
            * ex. 2026-02-26T19:35:19
        * station prediction time
            * timestamp for when _prediction_ was generated
            * `response['ctatt']['route'][0]['train'][0]['prdt']`
            * ex. 2026-02-26T19:33:19
        * is station delayed
            * bool to denote if tain is delayed or not
            * `response['ctatt']['route'][0]['train'][0]['isDly']`
        * is app
            * bool to denote app showing "approaching" "due"
            `response['ctatt']['route'][0]['train'][0]['isApp']`
---
additional col locations 
* station direction ID, TrDr
    * numeric code that translates to direction
    * ex. for red line 1 is howard bound 5 is 95th bound
    * codes will only be 1 **or** 5
    * `response['ctatt']['route'][0]['train'][0]['trDr']`
* cardinal direction
    * N, S, E, W
        * `response['ctatt']['route'][0]['train'][0]['heading']`
* train line name
    * have each train line map to an ID and store ID to train @name in seperate db table?
    * `response['ctatt']['route'][0]['@name']`



        * next station name
            * ex. Kedzie
            * `response['ctatt']['route'][0]['train'][0]['nextStaNm']`
        * after df is built and clean dtypes load it into local postgres db
            * `pd.to_sql("table_name", if_exists=append, index=False, con=con)`
            * `conn = psycopg2.connect(
                dbname="school",
                user="postgres",
                password="your_password",
                host="localhost")`
                * https://www.geeksforgeeks.org/postgresql/postgresql-connecting-to-the-database-using-python/
---

#### data model

Database >> schema >> tables
* schemas: raw/stage & fact/dim
    * raw/stage: for csv dim table and raw load of lapi fact table
        * for initial load of models [tables]
        * materiazlie as view
    * fact/dim: output of  transformations
        * materiazlie as table
        * dev schema?


1. train_locations: fact table from lapi
    * cols: `['unique_primary_key', 'next_station_parent_id',
           'next_stop_id', 'arrival_time',
           'prediction_time',
           'is_app', 'is_delayed' ]`
    * primary key: unique primary key when data is written to table
    * foreign keys: `next_stop_id`
    * data transformations: 
        * create primary key
        * `next_station_parent_id` & `next_stop_id` as int
        * `arrival_time` & `prediction_time` read as time stamps
        * `is_app` & `is_delayed` as bool

2. stations: dimension table from csv
    * cols: `['DIRECTION_ID', 'STOP_NAME', 'STATION_NAME', 'STATION_DESCRIPTIVE_NAME',
       'MAP_ID', 'ADA', 'RED', 'BLUE', 'G', 'BRN', 'P', 'Y', 'Pnk', 'O',
       'Location']`
    * primary key: `STOP_ID`
    * foreign keys: none
    * data transformations: rename `stop_id` or `next_stop_id` so col names match, lower case col names

* https://www.tecmint.com/install-postgresql-with-pgadmin4-on-linux-mint/
---

#### DBT Notes


In [19]:
my_col_name = ['train']
my_data = [['orange', 'red', 'blue']]
my_dict = dict(zip(my_col_name, my_data))
my_df = pd.DataFrame(my_dict)

my_df.head()

,train
0,orange
1,red
2,blue


In [29]:
my_test_1 =[1, 2, 3]
my_test_2 = [2, 3, 4]

combine_lists = []
combine_lists.append([my_test_2, my_test_1])
# combine_lists.append(my_test_1)

print(combine_lists)

[[[2, 3, 4], [1, 2, 3]]]


In [ ]:
def get_half_trns_lapi_call(is_first_half_rt_nms: bool) -> dict[int, str]:

    CTA_API_KEY = os.getenv('CTA_API_KEY')

    rt_names = ["Red", "Blue", "Brn", "G", "Org", "P", "Pink", "Y"]

    
    if is_first_half_rt_nms==1: 
        location_url = ''.join(['http://lapi.transitchicago.com/api/1.0/ttpositions.aspx?key='
                                ,CTA_API_KEY
                                ,'&rt=', rt_names[0]
                                ,'&rt=', rt_names[1]
                                ,'&rt=', rt_names[2]    # api has max of four train lines at once
                                ,'&rt=', rt_names[3]
                                ,'&outputType=JSON'])

    else:
        location_url = ''.join(['http://lapi.transitchicago.com/api/1.0/ttpositions.aspx?key='
                                ,CTA_API_KEY
                                ,'&rt=', rt_names[-4]
                                ,'&rt=', rt_names[-3]
                                ,'&rt=', rt_names[-2]
                                ,'&rt=', rt_names[-1]
                                ,'&outputType=JSON'])
        

    response = requests.get(location_url).json()


    stop = 0
    train = 0
    rt_len = len(response['ctatt']['route'])
    all_nxt_sta_dict= {} # {stationId: station_name}


    for train in range(rt_len):
        
        for stop in range(len(response['ctatt']['route'][train]['train'])): # active stops on train line
                all_nxt_sta_dict[int(response['ctatt']['route'][train]['train'][stop]['nextStaId'])] = response['ctatt']['route'][train]['train'][stop]['nextStaNm']
                stop+=1

        train+=1

        
    return all_nxt_sta_dict

In [53]:
get_half_trns_lapi_call(is_first_half_rt_lst=False)

{40310: 'Western',
 41130: 'Halsted',
 40040: 'Quincy',
 40930: 'Midway',
 40960: 'Pulaski',
 40900: 'Howard',
 40400: 'Noyes',
 40740: 'Western',
 40170: 'Ashland',
 41030: 'Polk',
 40850: 'Harold Washington Library-State/Van Buren',
 41680: 'Oakton-Skokie'}

In [54]:
get_half_trns_lapi_call(is_first_half_rt_lst=True)

{41000: 'Cermak-Chinatown',
 40450: '95th/Dan Ryan',
 41490: 'Harrison',
 41420: 'Addison',
 41170: 'Garfield',
 41300: 'Loyola',
 40650: 'North/Clybourn',
 41220: 'Fullerton',
 41400: 'Roosevelt',
 40080: 'Sheridan',
 40760: 'Granville',
 41320: 'Belmont',
 41450: 'Chicago',
 40910: '63rd',
 40470: 'Racine',
 40010: 'Austin',
 40250: 'Kedzie-Homan',
 40670: "Western (O'Hare Branch)",
 40750: "Harlem (O'Hare Branch)",
 41330: 'Montrose',
 41410: 'Chicago',
 41240: 'Addison',
 40980: 'Harlem (Forest Park Branch)',
 40810: 'Illinois Medical District',
 40800: 'Sedgwick',
 40660: 'Armitage',
 40680: 'Adams/Wabash',
 40090: 'Damen',
 40360: 'Southport',
 40710: 'Chicago',
 41480: 'Western',
 41700: 'Washington/Wabash',
 40020: 'Harlem/Lake',
 40290: 'Ashland/63rd',
 40130: '51st',
 41710: 'Damen',
 41120: '35th-Bronzeville-IIT',
 40480: 'Cicero',
 40700: 'Laramie',
 40170: 'Ashland'}

* need to do another API call or just download file [explore + research pros and cons] to get all CTA station stops to map to all GPIO pins
* probably only do call once when running map and store data but continue to run `get_half_trns_lapi_call` to get up to date train location info

In [ ]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
from sodapy import Socrata

# `https://data.cityofchicago.org/api/v3/views/8pix-ypme/query.json`
https://data.cityofchicago.org/resource/8pix-ypme.json
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofchicago.org", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cityofchicago.org,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("8pix-ypme", limit=2000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
